* Colab jupyter notebook끼리 import하기 위한 선언

In [48]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1


In [49]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

In [50]:
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1/sample')

In [3]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1/sample']

* 계산 그래프란? 계산과정을 그래프로 나타낸 것. 우리가 잘아는 그래프 자료구조로 노드와 엣지(선)로 표현됨
* 순전파 : 계산 그래프의 출발점부터 종착점으로의 전파
* 역전파 :
* 계산 그래프로 푸는 이유 ?

1. 국소적 계산 : 전체가 아무리복잡해도 각 노드에서는 단순한 계산에 집중해 문제를 단순화 할 수 있음
2. 중간 계산 결과 모두 보관 
3. 역전파를 통해 '미분'을 효율적으로 계산할 수 있음
* 역전파는 '국소적인 미분'을 순방향과 반대 방향에서 전달하며, 이 '국소적인 미분'을 전달하는 원리는 연쇄법칙에 따른다.


In [8]:
# 곱셈 계층
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x*y
    return out
  
  def backward(self, dout):
    # dout => 상류에서 넘어온 미분(dout)
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

In [20]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [21]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [13]:
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y

    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    
    return dx, dy

In [23]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(price)

715.0000000000001


In [24]:
# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(dapple, dapple_num, dorange, dorange_num, dtax)

2.2 110.00000000000001 3.3000000000000003 165.0 650


In [41]:
# ReLU
class ReLU:
  def __init__(self):
    self.mask = None
  
  def forward(self, x):
    self.mask = (x<=0)
    out = x.copy()
    out[self.mask] = 0
    
    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout

    return dx

In [42]:
x = np.array([[1.0, -0.5],[-2.0, 3.0]])
x

array([[ 1. , -0.5],
       [-2. ,  3. ]])

In [43]:
mask = (x<=0)
mask

array([[False,  True],
       [ True, False]])

In [44]:
x[mask] = 0
x

array([[1., 0.],
       [0., 3.]])

In [45]:
class Sigmoid:
  def __init__(self):
    self.out = None

  def forward(self, x):
    out = 1 /(1+np.exp(-x))
    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

* Affine 계층 : 신경망의 순전차에선 가중치 신호의 총합을 계산하기 때문에 행렬의 곱을 사용했음

In [38]:
X_dot_W = np.array([[0, 0, 0],[10, 10, 10]])
B = np.array([1,2,3])

In [39]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [40]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [46]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
  
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    
    return out

  def backward(self, dout):
    dx = np.dot(dout. self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    return dx

In [55]:
class SoftWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)

    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size

    return dx

In [53]:
#미니배치 학습 구현
import numpy as np
from sample.dataset_py import mnist
from sample.common import *
from sample.common.gradient import numerical_gradient
from collections import OrderedDict

In [56]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size,
               weight_init_std=0.01):
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)

    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)

    self.params['b2'] = np.zeros(output_size)

    # 계층 생성
    self.layers = OrderedDict()
    self.layers['Affen1'] = Affine(self.params['W1'], self.parmas['b1'])
    self.layers['Relu1'] = ReLU()
    self.layers['Affen2'] = Affine(self.params['W2'], self.parmas['b2'])
    self.lastLayer = SoftWithLoss()


  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)

    return x
  
  def loss(self, x, t):
    y = self.predict(x)

    return self.lastLayer(y,t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y==t) / float(x.shape[0])
    return accuracy
  
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x,t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

  def gradient(self, x, t):
    # 순전파
    self.loss(x, t)

    # 역전파
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db

    return grads

In [ ]:
(x_train, y_train),(x_test, y_test)=mnist.load_mnist(
    normalize=True, one_hot_label=True)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/ batch_size, 1)

for i in range(100): 
  # 미니배치 계산
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = y_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
  
  # 학습 기록
  loss = network.loss(x_batch, t_batch)
  print("Loss : {}".format(loss))
  train_loss_list.append(loss)

  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, y_train)
    test_acc = network.accuracy(x_test, y_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train acc, test acc | {}, {}".format(train_acc, test_acc))